<a href="https://colab.research.google.com/github/Aruminium/pdfConsolidation/blob/main/pdfConsolidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#インストール
!pip install PyPDF2
!pip install natsort
!pip install pycryptodome

In [57]:
import PyPDF2
import subprocess
import glob
from natsort import natsorted
import os
import re
from Crypto.Cipher import AES
import unicodedata
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title 準備 { vertical-output: true }

#@markdown **googleドライブのパス(基本的に変更しない)**
gdrivePath = '/content/drive/MyDrive/' #@param {type:"string"}

#@markdown **連結させたいpdfが在るディレクトリのパス**
dirPath = '' #@param {type:"string"}

#連結するpdfをまとめたディレクトリのパス
files = glob.glob(gdrivePath + dirPath + "*.pdf")

# 全角->半角
[os.rename(pdfFile, unicodedata.normalize("NFKC", pdfFile)) for pdfFile in files]

#連結するpdfをまとめたディレクトリのパス
files = glob.glob(gdrivePath + dirPath + "*.pdf")

fileNames = [os.path.basename(pdfFile).replace(".pdf", "") for pdfFile in files]
fileNames

In [ ]:
#@title 並び替え　{ vertical-output: true }

sorting = "ascending" #@param ["unsort", "ascending", "descending", "reversion"]

if sorting in "ascending":
    files = natsorted(files)
elif sorting in "descending":
    files = natsorted(files).reverse()
elif sorting in "reversion":
    files = files.reverse()

#確認
fileNames = [os.path.basename(pdfFile).replace(".pdf", "") for pdfFile in files]
fileNames

In [ ]:
#@title ページ数取得　{ vertical-output: true }
pdfPages = []
sumPages = 0
for pdfFile in files:
    pdf_reader = PyPDF2.PdfFileReader(pdfFile)
    page_num = pdf_reader.getNumPages()
    pdfPages.append(sumPages)
    sumPages += page_num
    print(sumPages)
pdfPages

In [ ]:

#@title 出力　{ vertical-output: true }

#@markdown **ファイル名**
pdfName = '' #@param {type:"string"}
#@markdown **MyDrive/以下の出力path**
exportPath = '' #@param {type:"string"}

pdf_writer = PyPDF2.PdfFileMerger()

for file in files:
    pdf_writer.append(file)

#出力パス gdrivePath + "/ + 任意ディレクトリのpath(/のみでも大丈夫)" + fileName
outPath = gdrivePath + exportPath + pdfName + '.pdf'

# exportPathがなければ作る
if not os.path.exists(exportPath):
  os.mkdir(exportPath)

pdf_writer.write(outPath)
pdf_writer.close()

In [100]:
#@title しおり作成 & 圧縮

#@markdown うまくいかない時は1つ前のセルのpdf結合からやり直す

writer = PyPDF2.PdfFileWriter()
reader = PyPDF2.PdfFileReader(outPath)

for i in range(reader.getNumPages()):
    #圧縮
    page = reader.getPage(i)
    page.compressContentStreams()
    writer.addPage(page)

for key, value in zip(fileNames, pdfPages):
    writer.addBookmark(key, value, parent=None)
writer.setPageMode("/UseOutlines")  # This is what tells the PDF to open to bookmarks
with open(outPath, "wb") as fp:
    writer.write(fp)